# DeepRank-Core basic Protein-Protein Interface training

## Setup

### Data

The example data used in this tutorial are available on [tdb, zenodo?](). To download the data, please [visit the dataset page for this workshop and click the “Download all” button. Unzip the downloaded file, and] save the contents as a folder called `data/` in the same directory of this notebook. (The name data and the folder location are optional but recommended, as this are the name and the location we will use to refer to the folder throughout the tutorial.)

The dataset contains only 100 data points, which are obviously not enough to develop an impactful predictive model, and the scope of its use is indeed only demonstrative and informative for the users.

### Software

1. Follow the [updated instructions](https://github.com/DeepRank/deeprank-core#installation) in the README.md on the main branch for successfully installing `deeprankcore` package.
2. To test the environment in which` deeprankcore` has been (successfully) installed, first clone [deeprank-core repository](https://github.com/DeepRank/deeprank-core). Navigate into it, and after having activated the environment and installed [pytest](https://anaconda.org/anaconda/pytest), run `pytest tests`. All tests should pass. We recommend installing `deeprankcore` and all its dependencies into a [conda](https://docs.conda.io/en/latest/) environment.
3. Additionally, for this tutorial you need to install [scikit-learn](https://scikit-learn.org/stable/install.html).

## Introduction

<img style="margin-left: 1.5rem" align="right" src="images/training_ppi.png" width="400">

This tutorial will demonstrate the use of DeepRank-Core for training Graph Neural Networks (GNNs) and Convolutional Neural Networks (CNNs) using Protein-Protein Interface (PPI) data for classification and regression predictive tasks.

This tutorial assumes that the PPI data of interest have already been generated and saved into [HDF5 files](https://en.wikipedia.org/wiki/Hierarchical_Data_Format), with the data structure that DeepRank-Core expects. Such data have already been generated in `data/ppi/processed/` folder for the user, but for more details about the process please refer to the tutorial notebook [data_generation_ppi.ipynb](insert_link). It contains a detailed description of how such data are generated starting from the PDB files and gives instructions for generating different resolutions' data, i.e. residue- and atomic-level.

This tutorial assumes also a basic knowledge of the [PyTorch](https://pytorch.org/) framework, on top of which the machine learning pipeline of DeepRank-Core has been developed. 

### Use case

<img style="margin-right: 1.5rem" align="left" src="images/pmhc_pdb_example.png" width="200"/>

The example dataset that we provide contains PDB files representing [Major Histocompatibility Complex (MHC) protein](https://en.wikipedia.org/wiki/Major_histocompatibility_complex) + peptide (pMHC) complexes, which play a key role in T-cell immunity. We are interested in predicting the Binding Affinity (BA) of the complexes, which can be used to determine the most suitable mutated tumor peptides as vaccine candidates.

PDB models used in this tutorial have been generated with [PANDORA](https://github.com/X-lab-3D/PANDORA), an anchor restrained modeling pipeline for generating peptide-MHC structures. While target data, so the BA values for such pMHC complexes, have been retrieved from [MHCFlurry 2.0](https://data.mendeley.com/datasets/zx3kjzc3yx).

On the left an example of a pMHC structure is shown, rendered using [ProteinViewer](https://marketplace.visualstudio.com/items?itemName=ArianJamasb.protein-viewer). The MHC protein is displayed in green, while the peptide is in orange.

## Utilities

### Libraries

Let's import the libraries needed for this tutorial:

In [9]:
import glob
import os
import h5py
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import numpy as np
np.seterr(divide = 'ignore')
np.seterr(invalid='ignore')
from deeprankcore.dataset import GraphDataset
from deeprankcore.trainer import Trainer
from deeprankcore.neuralnets.gnn.naive_gnn import NaiveNetwork
from deeprankcore.utils.exporters import HDF5OutputExporter

### Paths and sets

Let's define the paths for reading the processed data:

In [22]:
level = "residue"
processed_data_path = os.path.join("data", "ppi", "processed", level)
input_data_path = glob.glob(os.path.join(processed_data_path, '*.hdf5'))
output_path = os.path.join("data", "ppi") # for saving predictions results

The levels refer to a different molecular resolution, either residue- or atomic-level. In residue-level graphs, each node represents one amino acid residue, while in atomic-level graphs each node represents one atom within the amino acid residues. In this tutorial, we will use residue-level data, but the same code could be applied to atomic-level data with no changes.

Let's definea Pandas dataframe containing data points' IDs and the binary target values:

In [23]:
df_dict = {}
df_dict['entry'] = []
df_dict['target'] = []
for fname in input_data_path:
    with h5py.File(fname, 'r') as hdf5:
        for mol in hdf5.keys():
            target_value = float(hdf5[mol]["target_values"]["binary"][()])
            df_dict['entry'].append(mol)
            df_dict['target'].append(target_value)

df = pd.DataFrame(data=df_dict)
df.head()

,entry,target
0,residue-ppi:M-P:BA-102611,1.0
1,residue-ppi:M-P:BA-102669,0.0
2,residue-ppi:M-P:BA-102719,0.0
3,residue-ppi:M-P:BA-114468,0.0
4,residue-ppi:M-P:BA-115138,0.0


As explained in [data_generation_ppi.ipynb tutorial](insert_link), for each data point there are two targets: "BA" and "binary". The first represents the continuous Binding Affinity value of the complex, while the second represents its binary representation, being 0 (BA > 500 nM) a not-binding complex and 1 (BA <= 500 nM) binding one.

The dataframe `df` is used only to split data points into training, validation and test sets according to the "binary" target - using the target stratification, to keep the proportion of 0s and 1s sort of constant among the different sets. Training and validation set will be used during the training for updating the netowork's weights, while the test set will be held out as an indipendent test and will be used for later model's evaluation.

In [24]:
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.target, random_state=42)
df_train, df_valid = train_test_split(df_train, test_size=0.2, stratify=df_train.target, random_state=42)

print(f'Data statistics:\n')
print(f'Total samples: {len(df)}\n')
print(f'Training set: {len(df_train)} samples, {round(100*len(df_train)/len(df))}%')
print(f'\t- Class 0: {len(df_train[df_train.target == 0])} samples, {round(100*len(df_train[df_train.target == 0])/len(df_train))}%')
print(f'\t- Class 1: {len(df_train[df_train.target == 1])} samples, {round(100*len(df_train[df_train.target == 1])/len(df_train))}%')
print(f'Validation set: {len(df_valid)} samples, {round(100*len(df_valid)/len(df))}%')
print(f'\t- Class 0: {len(df_valid[df_valid.target == 0])} samples, {round(100*len(df_valid[df_valid.target == 0])/len(df_valid))}%')
print(f'\t- Class 1: {len(df_valid[df_valid.target == 1])} samples, {round(100*len(df_valid[df_valid.target == 1])/len(df_valid))}%')
print(f'Testing set: {len(df_test)} samples, {round(100*len(df_test)/len(df))}%')
print(f'\t- Class 0: {len(df_test[df_test.target == 0])} samples, {round(100*len(df_test[df_test.target == 0])/len(df_test))}%')
print(f'\t- Class 1: {len(df_test[df_test.target == 1])} samples, {round(100*len(df_test[df_test.target == 1])/len(df_test))}%')

Data statistics:

Total samples: 100

Training set: 72 samples, 72%
	- Class 0: 36 samples, 50%
	- Class 1: 36 samples, 50%
Validation set: 18 samples, 18%
	- Class 0: 9 samples, 50%
	- Class 1: 9 samples, 50%
Testing set: 10 samples, 10%
	- Class 0: 5 samples, 50%
	- Class 1: 5 samples, 50%


## Classification

Let's train a GNN and a CNN for a classification predictive task, which consists in predicting the "binary" target values. 

### GNN

#### GraphDataset

For training GNNs the user can create `GraphDataset` instances. The latter class inherits from `DeeprankDataset` class, which in turns inherits from `Dataset` [PyTorch geometric class](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/data/dataset.html), a base class for creating graph datasets.

A few notes about `GraphDataset` parameters:
- By default, all features contained in the HDF5 files are used, but the user can specify `node_features` and `edge_features` in `GraphDataset` if not all of them are needed. See the [docs](https://deeprankcore.readthedocs.io/en/latest/features.html) for more details about all the possible pre-implemented features. 
- For the `GraphDataset` class only it is possible to define a dictionary to indicate which transformations to apply to the features, being the transformations lambda functions and/or standardization. If `True`, standardization is applied after transformation, if the latter is present. Standardization consists in applying the following formula on each feature's value: ${x' = \frac{x - \mu}{\sigma}}$, being ${\mu}$ the mean and ${\sigma}$ the standard deviation. Standardization is a scaling method where the values are centered around mean with a unit standard deviation. In the example below we will apply a logarithmic transformation and then standardization to all the features. It is also possible to use specific features as keys for indicating that we want to apply transformation and/or standardization to few features only. 
- Since we are applying standardization, we need to use training features' means and standard deviations to scale validation and test sets. For doing so, `train` and `dataset_train` parameters are used. When `train` is set `False`, a `dataset_train` of the same class must be provided and it will be used to scale the validation/testing set according to its features values. You need to pass `features_transform` to the training dataset only, since in other cases it will be ignored and only the one of `dataset_train` will be considered. 

In [26]:
target = "binary"
task = "classif"
features_transform = {'all': {'transform': lambda x: np.sqrt(x), 'standardize': True}}

print('Loading training data...')
dataset_train = GraphDataset(
    hdf5_path = input_data_path,
    subset = list(df_train.entry), # selects only data points with ids in df_train.entry
    target = target,
    task = task,
    features_transform = features_transform
)
print('Loading validation data...')
dataset_val = GraphDataset(
    hdf5_path = input_data_path,
    subset = list(df_valid.entry), # selects only data points with ids in df_valid.entry
    target = target,
    task = task,
    train = False,
    dataset_train = dataset_train
)
print('Loading test data...')
dataset_test = GraphDataset(
    hdf5_path = input_data_path,
    subset = list(df_test.entry), # selects only data points with ids in df_test.entry
    target = target,
    task = task,
    train = False,
    dataset_train = dataset_train
)

Loading training data...
   ['data/ppi/processed/residue/proc-28835.hdf5', 'data/ppi/processed/residue/proc-28842.hdf5', 'data/ppi/processed/residue/proc-28839.hdf5', 'data/ppi/processed/residue/proc-28838.hdf5', 'data/ppi/processed/residue/proc-28843.hdf5', 'data/ppi/processed/residue/proc-28834.hdf5', 'data/ppi/processed/residue/proc-28837.hdf5', 'data/ppi/processed/residue/proc-28840.hdf5', 'data/ppi/processed/residue/proc-28841.hdf5', 'data/ppi/processed/residue/proc-28836.hdf5'] dataset                 : 100%|██████████| 10/10 [00:00<00:00, 383.71it/s, entry_name=proc-28836.hdf5]
Loading validation data...
   ['data/ppi/processed/residue/proc-28835.hdf5', 'data/ppi/processed/residue/proc-28842.hdf5', 'data/ppi/processed/residue/proc-28839.hdf5', 'data/ppi/processed/residue/proc-28838.hdf5', 'data/ppi/processed/residue/proc-28843.hdf5', 'data/ppi/processed/residue/proc-28834.hdf5', 'data/ppi/processed/residue/proc-28837.hdf5', 'data/ppi/processed/residue/proc-28840.hdf5', 'data/ppi

features_transform parameter, if set, will be ignored.
features_transform will remain the same as the one used in training phase.



Loading test data...
   ['data/ppi/processed/residue/proc-28835.hdf5', 'data/ppi/processed/residue/proc-28842.hdf5', 'data/ppi/processed/residue/proc-28839.hdf5', 'data/ppi/processed/residue/proc-28838.hdf5', 'data/ppi/processed/residue/proc-28843.hdf5', 'data/ppi/processed/residue/proc-28834.hdf5', 'data/ppi/processed/residue/proc-28837.hdf5', 'data/ppi/processed/residue/proc-28840.hdf5', 'data/ppi/processed/residue/proc-28841.hdf5', 'data/ppi/processed/residue/proc-28836.hdf5'] dataset                 : 100%|██████████| 10/10 [00:00<00:00, 851.07it/s, entry_name=proc-28836.hdf5]

features_transform parameter, if set, will be ignored.
features_transform will remain the same as the one used in training phase.


#### Trainer

The class `Trainer` implements training, validation and testing of PyTorch-based neural networks. 

A few notes about `Trainer` parameters:
- `neuralnet` can be whatever neural network class which inherits from `torch.nn.Module`, and it shouldn't be specific to regression or classification in terms of output shape. `Trainer` class takes care of formatting the output shape according to the task. In this tutorial we will use `NaiveNetwork` (in `deeprankcore.neuralnets.gnn.naive_gnn`), whose architecture is shown below:
  ```python
  class NaiveConvolutionalLayer(Module):

    def __init__(self, count_node_features, count_edge_features):
        super().__init__()
        message_size = 32
        edge_input_size = 2 * count_node_features + count_edge_features
        self._edge_mlp = Sequential(Linear(edge_input_size, message_size), ReLU())
        node_input_size = count_node_features + message_size
        self._node_mlp = Sequential(Linear(node_input_size, count_node_features), ReLU())

    def forward(self, node_features, edge_node_indices, edge_features):
        # generate messages over edges
        node0_indices, node1_indices = edge_node_indices
        node0_features = node_features[node0_indices]
        node1_features = node_features[node1_indices]
        message_input = torch.cat([node0_features, node1_features, edge_features], dim=1)
        messages_per_neighbour = self._edge_mlp(message_input)
        # aggregate messages
        out = torch.zeros(node_features.shape[0], messages_per_neighbour.shape[1]).to(node_features.device)
        message_sums_per_node = scatter_sum(messages_per_neighbour, node0_indices, dim=0, out=out)
        # update nodes
        node_input = torch.cat([node_features, message_sums_per_node], dim=1)
        node_output = self._node_mlp(node_input)
        return node_output

    class NaiveNetwork(Module):

        def __init__(self, input_shape: int, output_shape: int, input_shape_edge: int):
            """
            Args:
                input_shape (int): Number of node input features.
                output_shape (int): Number of output value per graph.
                input_shape_edge (int): Number of edge input features.
            """
            super().__init__()
            self._external1 = NaiveConvolutionalLayer(input_shape, input_shape_edge)
            self._external2 = NaiveConvolutionalLayer(input_shape, input_shape_edge)
            hidden_size = 128
            self._graph_mlp = Sequential(Linear(input_shape, hidden_size), ReLU(), Linear(hidden_size, output_shape))

        def forward(self, data):
            external_updated1_node_features = self._external1(data.x, data.edge_index, data.edge_attr)
            external_updated2_node_features = self._external2(external_updated1_node_features, data.edge_index, data.edge_attr)
            means_per_graph_external = scatter_mean(external_updated2_node_features, data.batch, dim=0)
            graph_input = means_per_graph_external
            z = self._graph_mlp(graph_input)
            return z
  ```
- `class_weights` is used in classification tasks only, to assign class weights based on the training dataset content.
- `cuda` and `ngpu` are used for indicating whether to use CUDA and how many GPUs. By default, CUDA is not used and `ngpu` is 0.

In [30]:
class_weights = True # weighted loss function

trainer = Trainer(
    neuralnet = NaiveNetwork,
    dataset_train = dataset_train,
    dataset_val = dataset_val,
    dataset_test = dataset_test,
    class_weights = class_weights,
    output_exporters = [HDF5OutputExporter(output_path)]
)

The default optimizer is `torch.optim.Adam`. It is possible to specify optimizer's parameters or to use another PyTorch optimizer object:

In [31]:
optimizer = torch.optim.SGD
lr = 1e-3
weight_decay = 0.001

trainer.configure_optimizers(optimizer, lr, weight_decay)

The default loss function for classification is `torch.nn.CrossEntropyLoss`. It is also possible to set other PyTorch loss functions by using `Trainer.set_lossfunction` method.

Then we can train our model, using the `train()` method of the `Trainer` class.

A few notes about `train()` method parameters:
- `earlystop_patience`, `earlystop_maxgap` and `min_epoch` are used for controlling early stopping logic. `earlystop_patience` indicates the number of epochs after which the training ends if the validation loss as not improved. `earlystop_maxgap` indicated the maximum difference allowed between validation and training loss, and `min_epoch` is the minimum epoch to be reached before looking at maxgap.
- `validate` is set True to perform validation on an independent dataset, which we called `dataset_val` few cells above.
- `num_workers` can be set for indicating how many subprocesses to use for data loading. The default is 0 and it means that the data will be loaded in the main process.

In [32]:
epochs = 20
batch_size = 8
earlystop_patience = 5
earlystop_maxgap = 0.1
min_epoch = 10

trainer.train(
    nepoch = epochs,
    batch_size = batch_size,
    earlystop_patience = earlystop_patience,
    earlystop_maxgap = earlystop_maxgap,
    min_epoch = min_epoch,
    validate = True,
    filename = os.path.join(output_path, 'model.pth.tar'))

epoch = trainer.epoch_saved_model
print(f"Model saved at epoch {epoch}")
pytorch_total_params = sum(p.numel() for p in trainer.model.parameters())
print(f'Total # of parameters: {pytorch_total_params}')
pytorch_trainable_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)
print(f'Total # of trainable parameters: {pytorch_trainable_params}')

/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/deeprank-core/deeprankcore/utils/exporters.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['phase', 'entry', 'output'], dtype='object')]

  self.df.to_hdf(


UnboundLocalError: local variable 'checkpoint_model' referenced before assignment

add test comment

In [ ]:
trainer.test(batch_size = batch_size, num_workers = num_workers)

add plot loss, e printa auc, f1 score

### CNN

## Regression

### GNN

### CNN